In [8]:
import json
import sys
from pathlib import Path

from easydict import EasyDict as edict

In [11]:
sys.path.append('..')
%cd ..

/home/ubuntu/VAST


/home/ubuntu/.cache/pypoetry/virtualenvs/libvast-Ju6CRI22-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
from libvast.model.vast import VAST

In [13]:
model_cfg = edict(json.load(Path('./config/vast/default_model_cfg.json').open()))
model_cfg.max_vision_sample_num = 1 # TODO
model_cfg.max_audio_sample_num = 1 # TODO
model_cfg

{'model_type': 'vast',
 'itm_ratio': 0.1,
 'frozen_vision': False,
 'frozen_audio': False,
 'checkpointing': False,
 'max_caption_len': 40,
 'max_omni_caption_len': 70,
 'max_subtitle_len': 70,
 'contra_dim': 512,
 'inherit_keys': ['vision_encoder_type',
  'audio_encoder_type',
  'audio_melbins',
  'audio_target_length'],
 'frame_embedding_type': 'adaptive',
 'vision_resolution': 224,
 'vision_encoder_type': 'evaclip01_giant',
 'audio_encoder_type': 'beats',
 'audio_melbins': 64,
 'audio_target_length': 1024,
 'beam_size': 3,
 'captioner_mode': False,
 'generate_nums': 1,
 'ret_bidirection_evaluation': False,
 'itm_rerank_num': 50,
 'evaluation_type': 'evaluation_mm',
 'max_vision_sample_num': 1,
 'max_audio_sample_num': 1}

In [29]:
# Video loading
sample_num = 5
vision_resolution = 224
vision_encoder_type = "evaclip01_giant"
video_path = "dance.mp4"

import decord
import torch
from torchvision.transforms.transforms import *

from libvast.utils.tool import split

if vision_encoder_type.startswith("clip") or vision_encoder_type.startswith("evaclip"):
    mean = [0.48145466, 0.4578275, 0.40821073]
    std = [0.26862954, 0.26130258, 0.27577711]
else:
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]


transforms = Compose(
    [
        Resize(vision_resolution),
        CenterCrop(vision_resolution),
        Normalize(mean, std),
    ]
)

container = decord.VideoReader(video_path)
print(len(container), container.get_avg_fps())
frames_ids = list(range(len(container)))
frames_splited = split(frames_ids, sample_num)
len(frames_splited), len(frames_splited[0])
frames_splited[1]
sample_idx = [i[(len(i) + 1) // 2 - 1] for i in frames_splited]
sample_idx
frames = container.get_batch(sample_idx).asnumpy()
frames.shape

frames = container.get_batch(sample_idx).asnumpy()
vision_pixels = torch.from_numpy(frames.transpose(0, 3, 1, 2) / 255.0)  ### nX3xHxW
vision_pixels = transforms(vision_pixels)
vision_pixels.shape

/home/ubuntu/.cache/pypoetry/virtualenvs/libvast-Ju6CRI22-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


748 25.0


torch.Size([5, 3, 224, 224])

In [ ]:
# inference
evaluation_dict = {}
feat_t = self.batch_get(batch,'feat_t')
evaluation_dict['feat_t'] = feat_t 
caption_tokens = self.batch_get(batch,'caption_tokens')
evaluation_dict['input_ids'] = caption_tokens.input_ids
evaluation_dict['attention_mask'] = caption_tokens.attention_mask
for task in subtasks:
#### compute_itc
assert task in ['tv','ta','tva','tvs','tvas']
feat_cond = self.batch_get(batch,f'feat_{task[1:]}')
evaluation_dict[f'feat_cond_{task}'] = feat_cond

condition_feats = self.batch_get(batch, f'condition_feats_{task[1:]}')
evaluation_dict[f'condition_feats_{task}'] = condition_feats
